In [1]:
import pandas as pd
import numpy as np

## 1) Only when using ExpoApp data: 
## Convert ExpoApp data to equidistant time series 

### 1.1) Scenario 1: Further analyses will be performed with the pa_proc pipeline ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_equidistant

# Create dataframe with equidistant acceleration in three axes
data_eq = expoapp_to_equidistant("C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/acc.csv", 1, "Europe/Madrid")

# Save dataframe to csv (optional)
data_eq.to_csv("C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/acc_eq.csv", index=False, header=False)

### 1.2) Scenario 2: Further analyses will be performed with GGIR ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_actigraph

# Create dataframe with equidistant acceleration in three axes, resembling an Actigraph file (to be used in GGIR)
data_act = expoapp_to_actigraph("C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/acc.csv",  1, "Europe/Madrid")

# Save dataframe to csv (optional)
data_act.to_csv("C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/acc_act.csv", index=False, header=False)

## 2) Convert raw acceleration data to Actigraph counts and ENMO

In [ ]:
from pa_proc.acc_to_intermediate import acc_to_counts_enmo

# Specify path of where data is saved
path = "C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/"

[enmo_10s, counts_enmo] = acc_to_intermediate(path + "acc_prep2.csv", 'expoapp', 30, 1)

## 3) calculate physical activity variables

In [6]:

# Identify different days in data
days_measured = counts_enmo.index.map(pd.Timestamp.date).unique()
 
# Create results dataframe
results = pd.DataFrame(index=days_measured)
 
# Calculate time in different PA domains (cut-offs used from document Gabriela)
for day_loop in days_measured:
	
	day_data = counts_enmo[(counts_enmo.index.date == day_loop)]
	
	results.loc[day_loop, 'weartime_choi'] = np.sum(day_data.wear_choi == 1)
	results.loc[day_loop, 'weartime_vanhees'] = np.sum(day_data.wear_vanhees == 1)
	
	day_data_wear_choi = day_data[day_data.wear_choi == 1]
	day_data_wear_vanhees = day_data[day_data.wear_vanhees == 1]
	
	results.loc[day_loop, 'counts_lpa'] = np.sum((day_data_wear_choi.counts_vm > 200) & (day_data_wear_choi.counts_vm <= 2690))
	results.loc[day_loop, 'counts_mvpa'] = np.sum(day_data_wear_choi.counts_vm > 2690)
	results.loc[day_loop, 'counts_vpa'] = np.sum(day_data_wear_choi.counts_vm > 6166)
 
 
	results.loc[day_loop, 'enmo_lpa'] = np.sum((day_data_wear_vanhees.enmo > 47.4) & (day_data_wear_vanhees.enmo < 69.1))
	results.loc[day_loop, 'enmo_mvpa'] = np.sum(day_data_wear_vanhees.enmo >= 69.1)
	results.loc[day_loop, 'enmo_vpa'] = np.sum(day_data_wear_vanhees.enmo >= 258.7)

results

,weartime_choi,weartime_vanhees,counts_lpa,counts_mvpa,counts_vpa,enmo_lpa,enmo_mvpa,enmo_vpa
2022-03-09,814.0,840.0,277.0,162.0,36.0,41.0,157.0,0.0
2022-03-10,752.0,780.0,300.0,157.0,44.0,47.0,161.0,3.0
2022-03-11,509.0,568.0,164.0,64.0,7.0,29.0,68.0,0.0
2022-03-12,215.0,257.0,62.0,2.0,0.0,9.0,1.0,0.0
2022-03-13,347.0,375.0,96.0,49.0,32.0,10.0,47.0,0.0
2022-03-14,217.0,315.0,74.0,72.0,30.0,5.0,72.0,2.0
2022-03-15,788.0,825.0,218.0,136.0,22.0,41.0,143.0,1.0
2022-03-16,754.0,795.0,195.0,116.0,5.0,20.0,114.0,0.0
2022-03-17,162.0,162.0,33.0,17.0,0.0,5.0,19.0,0.0
